# Your Title Here
askdlfjasdkfj
**Name(s)**: Brandon Dioneda, ASDF, qwer

**Website Link**: (your website link)

# QUESTION: Where and when do major power outages tend to occur?

## CODE

In [20]:
import pandas as pd
import numpy as np
import os

import plotly.express as px

pd.options.plotting.backend = "plotly"

In [21]:
import jupyter_black

jupyter_black.load()

### Data Cleaning	
Clean the data appropriately. For instance, you may need to replace data that should be missing with NaN or create new columns out of given ones (e.g. compute distances, scale data, or get time information from time stamps).	

In [22]:
# LOADING DATAFRAME
outage_fp = os.path.join("data", "outage.xlsx")
outage_raw = pd.read_excel(outage_fp, skiprows=5)

# DROPPING UNIT ROW AND NaN VARIABLES COLUMN
outage_p1 = outage_raw.drop(index=outage_raw.index[0], columns=["variables"])

# SETTING INDEX
outage_p2 = outage_p1.copy()
outage_p2.OBS = outage_p2.OBS.astype(int)
outage_p2.set_index("OBS", inplace=True)

# DATES
outage_p3 = outage_p2.copy()

outage_p3["OUTAGE.START"] = pd.to_datetime(
    outage_p2["OUTAGE.START.DATE"].astype(str)
    + " "
    + outage_p2["OUTAGE.START.TIME"].astype(str),
    errors="coerce",
)

outage_p3["OUTAGE.RESTORATION"] = pd.to_datetime(
    outage_p2["OUTAGE.RESTORATION.DATE"].astype(str)
    + " "
    + outage_p2["OUTAGE.RESTORATION.TIME"].astype(str),
    errors="coerce",
)

# CONVERSION AND DROPPING
outage_p4 = outage_p3.convert_dtypes(convert_string=False)
outage_p4.drop(
    columns=[
        "OUTAGE.START.DATE",
        "OUTAGE.START.TIME",
        "OUTAGE.RESTORATION.DATE",
        "OUTAGE.RESTORATION.TIME",
    ],
    inplace=True,
)

outage = outage_p4.copy()
outage

,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,CAUSE.CATEGORY,CAUSE.CATEGORY.DETAIL,...,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND,OUTAGE.START,OUTAGE.RESTORATION
OBS,,,,,,,,,,,,,,,,,,,,,
1,2011,7,Minnesota,MN,MRO,East North Central,-0.3,normal,severe weather,NaN,...,2279.0,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743,2011-07-01 17:00:00,2011-07-03 20:00:00
2,2014,5,Minnesota,MN,MRO,East North Central,-0.1,normal,intentional attack,vandalism,...,2279.0,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743,2014-05-11 18:38:00,2014-05-11 18:39:00
3,2010,10,Minnesota,MN,MRO,East North Central,-1.5,cold,severe weather,heavy wind,...,2279.0,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743,2010-10-26 20:00:00,2010-10-28 22:00:00
4,2012,6,Minnesota,MN,MRO,East North Central,-0.1,normal,severe weather,thunderstorm,...,2279.0,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743,2012-06-19 04:30:00,2012-06-20 23:00:00
5,2015,7,Minnesota,MN,MRO,East North Central,1.2,warm,severe weather,NaN,...,2279.0,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743,2015-07-18 02:00:00,2015-07-19 07:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1530,2011,12,North Dakota,ND,MRO,West North Central,-0.9,cold,public appeal,NaN,...,2192.2,1868.2,3.9,0.27,0.1,97.599649,2.401765,2.401765,2011-12-06 08:00:00,2011-12-06 20:00:00
1531,2006,<NA>,North Dakota,ND,MRO,West North Central,<NA>,NaN,fuel supply emergency,Coal,...,2192.2,1868.2,3.9,0.27,0.1,97.599649,2.401765,2.401765,NaT,NaT
1532,2009,8,South Dakota,SD,RFC,West North Central,0.5,warm,islanding,NaN,...,2038.3,1905.4,4.7,0.3,0.15,98.307744,1.692256,1.692256,2009-08-29 22:54:00,2009-08-29 23:53:00


### Univariate Analysis	
Look at the distributions of relevant columns separately by using DataFrame operations and drawing at least two relevant plots.	

In [60]:
px.histogram(
    outage["OUTAGE.START"].dt.hour,
    nbins=24,
    title="Outage Start Hour",
    labels={"value": "Hour of Day"},
)

# 

In [65]:
px.histogram(
    outage["OUTAGE.RESTORATION"].dt.hour,
    nbins=24,
    title="Outage Restoration Hour",
    labels={"value": "Hour of Day"},
)

In [25]:
px.histogram(outage, x="OUTAGE.DURATION", labels={"OUTAGE.DURATION": "Duration (min)"})

In [26]:
# For the 'Total Outages per State (year)' graph:
states = (
    outage.groupby(["YEAR", "U.S._STATE", "POSTAL.CODE"])[["CAUSE.CATEGORY"]]
    .count()
    .reset_index()
)
states.rename(columns={"CAUSE.CATEGORY": "OUTAGES"}, inplace=True)
states

,YEAR,U.S._STATE,POSTAL.CODE,OUTAGES
0,2000,Alabama,AL,3
1,2000,Alaska,AK,1
2,2000,Arizona,AZ,2
3,2000,California,CA,1
4,2000,Delaware,DE,1
...,...,...,...,...
400,2016,Tennessee,TN,2
401,2016,Texas,TX,6
402,2016,Utah,UT,5
403,2016,Washington,WA,7


In [27]:
outage["U.S._STATE"].value_counts()

California              210
Texas                   127
Washington               97
Michigan                 95
New York                 71
Maryland                 58
Pennsylvania             57
Illinois                 46
Florida                  45
Ohio                     43
Indiana                  43
Utah                     41
Delaware                 41
North Carolina           40
Louisiana                40
Virginia                 37
New Jersey               35
Tennessee                34
Arizona                  28
Oregon                   26
Arkansas                 25
Oklahoma                 24
Wisconsin                20
Maine                    19
Massachusetts            18
Connecticut              18
Georgia                  17
Missouri                 17
Colorado                 15
Minnesota                15
New Hampshire            14
Kentucky                 13
District of Columbia     10
Idaho                     9
Vermont                   9
Kansas              

In [67]:
outage["CAUSE.CATEGORY"].value_counts()

severe weather                   763
intentional attack               418
system operability disruption    127
public appeal                     69
equipment failure                 60
fuel supply emergency             51
islanding                         46
Name: CAUSE.CATEGORY, dtype: int64

In [70]:
outage.groupby("U.S._STATE")["CAUSE.CATEGORY"].value_counts().to_frame().to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th></th>\n      <th>CAUSE.CATEGORY</th>\n    </tr>\n    <tr>\n      <th>U.S._STATE</th>\n      <th>CAUSE.CATEGORY</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th rowspan="2" valign="top">Alabama</th>\n      <th>severe weather</th>\n      <td>5</td>\n    </tr>\n    <tr>\n      <th>intentional attack</th>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Alaska</th>\n      <th>equipment failure</th>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th rowspan="4" valign="top">Arizona</th>\n      <th>intentional attack</th>\n      <td>18</td>\n    </tr>\n    <tr>\n      <th>equipment failure</th>\n      <td>4</td>\n    </tr>\n    <tr>\n      <th>severe weather</th>\n      <td>4</td>\n    </tr>\n    <tr>\n      <th>system operability disruption</th>\n      <td>2</td>\n    </tr>\n    <tr>\n      <th rowspan="5" valign="top">Arkansas</th>\n      <th>severe 

In [73]:
outage.loc[
    (outage["CAUSE.CATEGORY"] == "intentional attack"),
    "CAUSE.CATEGORY.DETAIL",
].value_counts()

vandalism              335
sabotage                32
suspicious activity      3
Name: CAUSE.CATEGORY.DETAIL, dtype: int64

In [74]:
outage["CLIMATE.REGION"].value_counts()

Northeast             350
South                 229
West                  217
Central               200
Southeast             153
East North Central    138
Northwest             132
Southwest              92
West North Central     17
Name: CLIMATE.REGION, dtype: int64

In [62]:
from ipywidgets import widgets, interact, FloatSlider
from IPython.display import clear_output

DEFAULT = 2000
dropdown_cases = widgets.Dropdown(options=np.sort(states.YEAR.unique()), value=DEFAULT)


def yearly(year):
    """Displays amount of outages a state has from years 2000-2016"""
    outages_that_year = states[states.YEAR == year]
    fig = px.choropleth(
        outages_that_year,
        locations="POSTAL.CODE",
        locationmode="USA-states",
        scope="usa",
        color="OUTAGES",
        hover_name="U.S._STATE",
        color_continuous_scale=px.colors.sequential.Oranges,
        title=f"Total Outages per US State ({year})",
    )
    fig.show()


def dropdown_cases_handler(change):
    if change["name"] == "value" and (change["new"] != change["old"]):
        clear_output()
        display(dropdown_cases)
        yearly(change["new"])


display(dropdown_cases)
yearly(DEFAULT)
dropdown_cases.observe(dropdown_cases_handler)

Dropdown(options=(2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 20…

In [66]:
states_2 = (
    outage.groupby(["U.S._STATE", "POSTAL.CODE"])[["CAUSE.CATEGORY"]]
    .count()
    .reset_index()
)
states_2.rename(columns={"CAUSE.CATEGORY": "OUTAGES"}, inplace=True)

fig = px.choropleth(
    states_2,
    locations="POSTAL.CODE",
    locationmode="USA-states",
    scope="usa",
    color="OUTAGES",
    hover_name="U.S._STATE",
    color_continuous_scale=px.colors.sequential.Oranges,
    title=f"Total Outages per US State",
)
fig.show()

In [31]:
outage.dtypes

YEAR                              Int64
MONTH                             Int64
U.S._STATE                       object
POSTAL.CODE                      object
NERC.REGION                      object
CLIMATE.REGION                   object
ANOMALY.LEVEL                   Float64
CLIMATE.CATEGORY                 object
CAUSE.CATEGORY                   object
CAUSE.CATEGORY.DETAIL            object
HURRICANE.NAMES                  object
OUTAGE.DURATION                   Int64
DEMAND.LOSS.MW                    Int64
CUSTOMERS.AFFECTED                Int64
RES.PRICE                       Float64
COM.PRICE                       Float64
IND.PRICE                       Float64
TOTAL.PRICE                     Float64
RES.SALES                         Int64
COM.SALES                         Int64
IND.SALES                         Int64
TOTAL.SALES                       Int64
RES.PERCEN                      Float64
COM.PERCEN                      Float64
IND.PERCEN                      Float64


In [32]:
outage.columns

Index(['YEAR', 'MONTH', 'U.S._STATE', 'POSTAL.CODE', 'NERC.REGION',
       'CLIMATE.REGION', 'ANOMALY.LEVEL', 'CLIMATE.CATEGORY', 'CAUSE.CATEGORY',
       'CAUSE.CATEGORY.DETAIL', 'HURRICANE.NAMES', 'OUTAGE.DURATION',
       'DEMAND.LOSS.MW', 'CUSTOMERS.AFFECTED', 'RES.PRICE', 'COM.PRICE',
       'IND.PRICE', 'TOTAL.PRICE', 'RES.SALES', 'COM.SALES', 'IND.SALES',
       'TOTAL.SALES', 'RES.PERCEN', 'COM.PERCEN', 'IND.PERCEN',
       'RES.CUSTOMERS', 'COM.CUSTOMERS', 'IND.CUSTOMERS', 'TOTAL.CUSTOMERS',
       'RES.CUST.PCT', 'COM.CUST.PCT', 'IND.CUST.PCT', 'PC.REALGSP.STATE',
       'PC.REALGSP.USA', 'PC.REALGSP.REL', 'PC.REALGSP.CHANGE', 'UTIL.REALGSP',
       'TOTAL.REALGSP', 'UTIL.CONTRI', 'PI.UTIL.OFUSA', 'POPULATION',
       'POPPCT_URBAN', 'POPPCT_UC', 'POPDEN_URBAN', 'POPDEN_UC',
       'POPDEN_RURAL', 'AREAPCT_URBAN', 'AREAPCT_UC', 'PCT_LAND',
       'PCT_WATER_TOT', 'PCT_WATER_INLAND', 'OUTAGE.START',
       'OUTAGE.RESTORATION'],
      dtype='object')

### Bivariate Analysis

### ASSESSING MISSINGNESS



In [33]:
col_nan_counts = outage.isna().sum(axis=0)
col_nan_counts[col_nan_counts > 0]

MONTH                       9
CLIMATE.REGION              6
ANOMALY.LEVEL               9
CLIMATE.CATEGORY            9
CAUSE.CATEGORY.DETAIL     471
HURRICANE.NAMES          1462
OUTAGE.DURATION            58
DEMAND.LOSS.MW            705
CUSTOMERS.AFFECTED        443
RES.PRICE                  22
COM.PRICE                  22
IND.PRICE                  22
TOTAL.PRICE                22
RES.SALES                  22
COM.SALES                  22
IND.SALES                  22
TOTAL.SALES                22
RES.PERCEN                 22
COM.PERCEN                 22
IND.PERCEN                 22
POPDEN_UC                  10
POPDEN_RURAL               10
OUTAGE.START                9
OUTAGE.RESTORATION         58
dtype: int64

In [34]:
# If weekdays are important:
# outage['OUTAGE.START.WEEKDAY'] = (outage['OUTAGE.START']).dt.day_name()
# outage['OUTAGE.RESTORATION.WEEKDAY'] = (outage['OUTAGE.RESTORATION']).dt.day_name()
# outage

In [35]:
outage.drop(columns=["YEAR", "MONTH"]).describe()

,ANOMALY.LEVEL,OUTAGE.DURATION,DEMAND.LOSS.MW,CUSTOMERS.AFFECTED,RES.PRICE,COM.PRICE,IND.PRICE,TOTAL.PRICE,RES.SALES,COM.SALES,...,POPPCT_URBAN,POPPCT_UC,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND
count,1525.000000,1476.000000,829.000000,1.091000e+03,1512.000000,1512.000000,1512.000000,1512.000000,1.512000e+03,1.512000e+03,...,1534.000000,1534.000000,1534.000000,1524.000000,1524.000000,1534.000000,1534.000000,1534.000000,1534.000000,1534.000000
mean,-0.096852,2625.398374,536.287093,1.434562e+05,11.968373,10.135053,7.341468,10.117851,4.340337e+06,4.425831e+06,...,80.967112,9.545267,2594.174967,1558.041142,39.473491,8.604348,1.117608,90.158521,9.841405,2.911191
std,0.739957,5942.483307,2196.450393,2.869863e+05,3.088631,2.824150,2.473902,2.905234,3.377434e+06,3.497115e+06,...,11.900026,5.240315,1083.200091,313.149226,30.890074,11.134773,0.995424,10.519099,10.518892,2.115077
min,-1.600000,0.000000,0.000000,0.000000e+00,5.650000,4.700000,3.200000,4.700000,1.444170e+05,1.525170e+05,...,38.660000,0.000000,1232.600000,988.700000,0.400000,0.050000,0.000000,58.459995,0.240151,0.240151
25%,-0.500000,102.250000,3.000000,9.650000e+03,9.540000,8.017500,5.697500,7.957500,2.052570e+06,1.918971e+06,...,74.570000,5.220000,2016.300000,1307.800000,15.200000,3.350000,0.590000,86.382550,2.742036,1.730658
50%,-0.300000,701.000000,168.000000,7.013500e+04,11.460000,9.465000,6.710000,9.410000,3.441587e+06,3.182069e+06,...,84.050000,9.080000,2380.000000,1528.600000,29.500000,5.280000,0.970000,95.164177,4.835823,2.090873
75%,0.300000,2880.000000,400.000000,1.500000e+05,13.900000,11.340000,8.590000,11.740000,6.023744e+06,7.132397e+06,...,89.810000,12.020000,2847.425000,1732.200000,54.600000,8.680000,1.460000,97.258336,13.617450,3.645862
max,2.300000,108653.000000,41788.000000,3.241437e+06,34.580000,32.140000,27.850000,31.290000,1.862066e+07,1.404697e+07,...,100.000000,40.250000,9856.500000,2220.700000,142.300000,100.000000,6.210000,99.759849,41.540005,10.294118


In [36]:
# Correlation matrix:
corr = outage.corr()
corr

# corr = outage.corr().stack()
# corr = corr[corr.index.get_level_values(0) != corr.index.get_level_values(1)]
# corr

,YEAR,MONTH,ANOMALY.LEVEL,OUTAGE.DURATION,DEMAND.LOSS.MW,CUSTOMERS.AFFECTED,RES.PRICE,COM.PRICE,IND.PRICE,TOTAL.PRICE,...,POPPCT_URBAN,POPPCT_UC,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND
YEAR,1.000000,-0.145328,0.152132,-0.144047,-0.048470,-0.203106,0.285305,0.178094,0.107126,0.203980,...,-0.059175,0.133157,-0.049423,-0.035008,0.006746,0.002862,0.050914,0.083103,-0.083108,-0.065317
MONTH,-0.145328,1.000000,0.122687,0.019649,0.019242,0.044265,0.040726,0.029512,0.038572,0.032534,...,0.028650,-0.026817,0.028900,0.066761,-0.026504,-0.024165,-0.038223,-0.056710,0.056706,-0.004792
ANOMALY.LEVEL,0.152132,0.122687,1.000000,0.008910,0.060534,0.012768,-0.002123,-0.021202,-0.045939,-0.019087,...,0.042433,0.015403,0.016084,0.052256,-0.077142,-0.054828,-0.049914,0.032730,-0.032731,-0.038350
OUTAGE.DURATION,-0.144047,0.019649,0.008910,1.000000,0.026798,0.261916,0.004394,-0.001193,-0.040322,0.001722,...,-0.005116,-0.049927,0.022115,-0.002205,0.038422,0.025040,-0.025940,-0.121614,0.121605,0.030389
DEMAND.LOSS.MW,-0.048470,0.019242,0.060534,0.026798,1.000000,0.523746,0.016727,0.060988,0.034917,0.046997,...,0.051795,-0.076131,0.051451,0.012847,0.019149,0.035769,-0.018441,-0.037732,0.037734,0.021429
CUSTOMERS.AFFECTED,-0.203106,0.044265,0.012768,0.261916,0.523746,1.000000,-0.020290,-0.011285,-0.008099,-0.005548,...,0.113629,-0.117916,0.093878,0.060508,-0.052409,0.007171,-0.066623,-0.009530,0.009527,0.020321
RES.PRICE,0.285305,0.040726,-0.002123,0.004394,0.016727,-0.020290,1.000000,0.927087,0.813129,0.964819,...,0.280213,-0.332752,0.416170,0.271060,0.242106,0.259328,0.151521,-0.190775,0.190781,0.014160
COM.PRICE,0.178094,0.029512,-0.021202,-0.001193,0.060988,-0.011285,0.927087,1.000000,0.863704,0.977913,...,0.323708,-0.364091,0.492072,0.330239,0.200140,0.279196,0.092000,-0.154140,0.154139,0.033239
IND.PRICE,0.107126,0.038572,-0.045939,-0.040322,0.034917,-0.008099,0.813129,0.863704,1.000000,0.888185,...,0.302756,-0.349632,0.329301,0.258182,0.259206,0.326347,0.152045,-0.164432,0.164422,0.064923
TOTAL.PRICE,0.203980,0.032534,-0.019087,0.001722,0.046997,-0.005548,0.964819,0.977913,0.888185,1.000000,...,0.337433,-0.404706,0.472762,0.281729,0.254493,0.322697,0.136429,-0.178898,0.178897,0.083094


Major outages: depends on outage duration and # of people affected (at least 50,000 people)

## Code

In [37]:
nan_counts = outage.isna().sum(axis=0)
nan_counts[nan_counts > 0]

MONTH                       9
CLIMATE.REGION              6
ANOMALY.LEVEL               9
CLIMATE.CATEGORY            9
CAUSE.CATEGORY.DETAIL     471
HURRICANE.NAMES          1462
OUTAGE.DURATION            58
DEMAND.LOSS.MW            705
CUSTOMERS.AFFECTED        443
RES.PRICE                  22
COM.PRICE                  22
IND.PRICE                  22
TOTAL.PRICE                22
RES.SALES                  22
COM.SALES                  22
IND.SALES                  22
TOTAL.SALES                22
RES.PERCEN                 22
COM.PERCEN                 22
IND.PERCEN                 22
POPDEN_UC                  10
POPDEN_RURAL               10
OUTAGE.START                9
OUTAGE.RESTORATION         58
dtype: int64

In [38]:
nan_counts[nan_counts == 22].index

Index(['RES.PRICE', 'COM.PRICE', 'IND.PRICE', 'TOTAL.PRICE', 'RES.SALES',
       'COM.SALES', 'IND.SALES', 'TOTAL.SALES', 'RES.PERCEN', 'COM.PERCEN',
       'IND.PERCEN'],
      dtype='object')

In [39]:
# all columns w. 9 missing values are related to each other
# exactly 9 rows that match
outage[
    (outage["MONTH"].isna())
    | (outage["ANOMALY.LEVEL"].isna())
    | (outage["CLIMATE.CATEGORY"].isna())
    | (outage["OUTAGE.START"].isna())
]

,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,CAUSE.CATEGORY,CAUSE.CATEGORY.DETAIL,...,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND,OUTAGE.START,OUTAGE.RESTORATION
OBS,,,,,,,,,,,,,,,,,,,,,
240,2000,<NA>,Texas,TX,FRCC,South,<NA>,NaN,equipment failure,transformer outage,...,2435.3,1539.9,15.2,3.35,0.58,97.258336,2.742036,2.090873,NaT,NaT
340,2000,<NA>,Alabama,AL,SERC,Southeast,<NA>,NaN,severe weather,thunderstorm,...,1278.5,988.7,40.4,4.36,0.99,96.613888,3.386112,2.018314,NaT,NaT
366,2000,<NA>,Illinois,IL,SERC,Central,<NA>,NaN,severe weather,wildfire,...,2877.6,1759.5,28.6,7.11,1.12,95.864558,4.135442,1.415893,NaT,NaT
767,2000,<NA>,North Carolina,NC,SERC,Southeast,<NA>,NaN,severe weather,thunderstorm,...,1367.2,1043.7,73.5,9.48,2.11,90.336127,9.663873,7.52894,NaT,NaT
888,2000,<NA>,Delaware,DE,RFC,Northeast,<NA>,NaN,system operability disruption,NaN,...,1838.3,1083.0,97.3,20.88,6.21,78.30454,21.69546,3.656087,NaT,NaT
1319,2000,<NA>,Virginia,VA,SERC,Southeast,<NA>,NaN,equipment failure,relaying malfunction,...,2265.2,1179.2,53.3,6.75,0.97,92.320281,7.679719,2.997078,NaT,NaT
1507,2002,<NA>,Kansas,KS,SPP,South,<NA>,NaN,severe weather,winter storm,...,2176.5,1983.0,9.1,1.19,0.42,99.369212,0.632004,0.632004,NaT,NaT
1531,2006,<NA>,North Dakota,ND,MRO,West North Central,<NA>,NaN,fuel supply emergency,Coal,...,2192.2,1868.2,3.9,0.27,0.1,97.599649,2.401765,2.401765,NaT,NaT
1534,2000,<NA>,Alaska,AK,ASCC,NaN,<NA>,NaN,equipment failure,failure,...,1802.6,1276.0,0.4,0.05,0.02,85.761154,14.238846,2.901182,NaT,NaT


In [40]:
# confirmed that the columns that all have 22 missing values share the same 22 missing values

outage[
    outage["RES.PRICE"].isna()
    | outage["COM.PRICE"].isna()
    | outage["IND.PRICE"].isna()
    | outage["TOTAL.PRICE"].isna()
    | outage["RES.SALES"].isna()
    | outage["COM.SALES"].isna()
    | outage["IND.SALES"].isna()
    | outage["TOTAL.SALES"].isna()
    | outage["RES.PERCEN"].isna()
    | outage["COM.PERCEN"].isna()
    | outage["IND.PERCEN"].isna()
].to_excel("test.xlsx")

In [41]:
np.sum(outage.isna().sum(axis=1) > 0)

1493

### Hypothesis Testing

In [42]:
# TODO

In [43]:
import pandas as pd
import numpy as np
import os

import plotly.express as px

pd.options.plotting.backend = "plotly"

In [44]:
import jupyter_black

jupyter_black.load()

### Cleaning and EDA

In [45]:
# TODO
outage_fp = os.path.join("data", "outage.xlsx")
outage = pd.read_excel(
    outage_fp,
    skiprows=5,
)
outage

,variables,OBS,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,...,POPPCT_URBAN,POPPCT_UC,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND
0,Units,NaN,NaN,NaN,NaN,NaN,NaN,NaN,numeric,NaN,...,%,%,persons per square mile,persons per square mile,persons per square mile,%,%,%,%,%
1,NaN,1.0,2011.0,7.0,Minnesota,MN,MRO,East North Central,-0.3,normal,...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743
2,NaN,2.0,2014.0,5.0,Minnesota,MN,MRO,East North Central,-0.1,normal,...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743
3,NaN,3.0,2010.0,10.0,Minnesota,MN,MRO,East North Central,-1.5,cold,...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743
4,NaN,4.0,2012.0,6.0,Minnesota,MN,MRO,East North Central,-0.1,normal,...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1530,NaN,1530.0,2011.0,12.0,North Dakota,ND,MRO,West North Central,-0.9,cold,...,59.9,19.9,2192.2,1868.2,3.9,0.27,0.1,97.599649,2.401765,2.401765
1531,NaN,1531.0,2006.0,NaN,North Dakota,ND,MRO,West North Central,NaN,NaN,...,59.9,19.9,2192.2,1868.2,3.9,0.27,0.1,97.599649,2.401765,2.401765
1532,NaN,1532.0,2009.0,8.0,South Dakota,SD,RFC,West North Central,0.5,warm,...,56.65,26.73,2038.3,1905.4,4.7,0.3,0.15,98.307744,1.692256,1.692256
1533,NaN,1533.0,2009.0,8.0,South Dakota,SD,MRO,West North Central,0.5,warm,...,56.65,26.73,2038.3,1905.4,4.7,0.3,0.15,98.307744,1.692256,1.692256


In [46]:
# pd.concat([outage.iloc[0], outage.dtypes, outage.iloc[1:].isna().sum()], axis=1).iloc[2:]

In [47]:
outage.drop(index=outage.index[0], columns=["variables"], inplace=True)
outage.OBS = outage.OBS.astype(int)
outage.set_index("OBS", inplace=True)
# outage

In [48]:
outage["OUTAGE.START"] = pd.to_datetime(
    outage["OUTAGE.START.DATE"].astype(str)
    + " "
    + outage["OUTAGE.START.TIME"].astype(str),
    errors="coerce",
)
outage["OUTAGE.RESTORATION"] = pd.to_datetime(
    outage["OUTAGE.RESTORATION.DATE"].astype(str)
    + " "
    + outage["OUTAGE.RESTORATION.TIME"].astype(str),
    errors="coerce",
)

In [49]:
outage = outage.convert_dtypes(convert_string=False)
outage.drop(
    columns=[
        "OUTAGE.START.DATE",
        "OUTAGE.START.TIME",
        "OUTAGE.RESTORATION.DATE",
        "OUTAGE.RESTORATION.TIME",
    ],
    inplace=True,
)

In [50]:
outage

,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,CAUSE.CATEGORY,CAUSE.CATEGORY.DETAIL,...,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND,OUTAGE.START,OUTAGE.RESTORATION
OBS,,,,,,,,,,,,,,,,,,,,,
1,2011,7,Minnesota,MN,MRO,East North Central,-0.3,normal,severe weather,NaN,...,2279.0,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743,2011-07-01 17:00:00,2011-07-03 20:00:00
2,2014,5,Minnesota,MN,MRO,East North Central,-0.1,normal,intentional attack,vandalism,...,2279.0,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743,2014-05-11 18:38:00,2014-05-11 18:39:00
3,2010,10,Minnesota,MN,MRO,East North Central,-1.5,cold,severe weather,heavy wind,...,2279.0,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743,2010-10-26 20:00:00,2010-10-28 22:00:00
4,2012,6,Minnesota,MN,MRO,East North Central,-0.1,normal,severe weather,thunderstorm,...,2279.0,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743,2012-06-19 04:30:00,2012-06-20 23:00:00
5,2015,7,Minnesota,MN,MRO,East North Central,1.2,warm,severe weather,NaN,...,2279.0,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743,2015-07-18 02:00:00,2015-07-19 07:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1530,2011,12,North Dakota,ND,MRO,West North Central,-0.9,cold,public appeal,NaN,...,2192.2,1868.2,3.9,0.27,0.1,97.599649,2.401765,2.401765,2011-12-06 08:00:00,2011-12-06 20:00:00
1531,2006,<NA>,North Dakota,ND,MRO,West North Central,<NA>,NaN,fuel supply emergency,Coal,...,2192.2,1868.2,3.9,0.27,0.1,97.599649,2.401765,2.401765,NaT,NaT
1532,2009,8,South Dakota,SD,RFC,West North Central,0.5,warm,islanding,NaN,...,2038.3,1905.4,4.7,0.3,0.15,98.307744,1.692256,1.692256,2009-08-29 22:54:00,2009-08-29 23:53:00


In [51]:
outage.dtypes

YEAR                              Int64
MONTH                             Int64
U.S._STATE                       object
POSTAL.CODE                      object
NERC.REGION                      object
CLIMATE.REGION                   object
ANOMALY.LEVEL                   Float64
CLIMATE.CATEGORY                 object
CAUSE.CATEGORY                   object
CAUSE.CATEGORY.DETAIL            object
HURRICANE.NAMES                  object
OUTAGE.DURATION                   Int64
DEMAND.LOSS.MW                    Int64
CUSTOMERS.AFFECTED                Int64
RES.PRICE                       Float64
COM.PRICE                       Float64
IND.PRICE                       Float64
TOTAL.PRICE                     Float64
RES.SALES                         Int64
COM.SALES                         Int64
IND.SALES                         Int64
TOTAL.SALES                       Int64
RES.PERCEN                      Float64
COM.PERCEN                      Float64
IND.PERCEN                      Float64


In [52]:
outage.columns

Index(['YEAR', 'MONTH', 'U.S._STATE', 'POSTAL.CODE', 'NERC.REGION',
       'CLIMATE.REGION', 'ANOMALY.LEVEL', 'CLIMATE.CATEGORY', 'CAUSE.CATEGORY',
       'CAUSE.CATEGORY.DETAIL', 'HURRICANE.NAMES', 'OUTAGE.DURATION',
       'DEMAND.LOSS.MW', 'CUSTOMERS.AFFECTED', 'RES.PRICE', 'COM.PRICE',
       'IND.PRICE', 'TOTAL.PRICE', 'RES.SALES', 'COM.SALES', 'IND.SALES',
       'TOTAL.SALES', 'RES.PERCEN', 'COM.PERCEN', 'IND.PERCEN',
       'RES.CUSTOMERS', 'COM.CUSTOMERS', 'IND.CUSTOMERS', 'TOTAL.CUSTOMERS',
       'RES.CUST.PCT', 'COM.CUST.PCT', 'IND.CUST.PCT', 'PC.REALGSP.STATE',
       'PC.REALGSP.USA', 'PC.REALGSP.REL', 'PC.REALGSP.CHANGE', 'UTIL.REALGSP',
       'TOTAL.REALGSP', 'UTIL.CONTRI', 'PI.UTIL.OFUSA', 'POPULATION',
       'POPPCT_URBAN', 'POPPCT_UC', 'POPDEN_URBAN', 'POPDEN_UC',
       'POPDEN_RURAL', 'AREAPCT_URBAN', 'AREAPCT_UC', 'PCT_LAND',
       'PCT_WATER_TOT', 'PCT_WATER_INLAND', 'OUTAGE.START',
       'OUTAGE.RESTORATION'],
      dtype='object')

In [53]:
# If weekdays are important:
# outage['OUTAGE.START.WEEKDAY'] = (outage['OUTAGE.START']).dt.day_name()
# outage['OUTAGE.RESTORATION.WEEKDAY'] = (outage['OUTAGE.RESTORATION']).dt.day_name()
# outage

In [54]:
outage.drop(columns=["YEAR", "MONTH"]).describe()

,ANOMALY.LEVEL,OUTAGE.DURATION,DEMAND.LOSS.MW,CUSTOMERS.AFFECTED,RES.PRICE,COM.PRICE,IND.PRICE,TOTAL.PRICE,RES.SALES,COM.SALES,...,POPPCT_URBAN,POPPCT_UC,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND
count,1525.000000,1476.000000,829.000000,1.091000e+03,1512.000000,1512.000000,1512.000000,1512.000000,1.512000e+03,1.512000e+03,...,1534.000000,1534.000000,1534.000000,1524.000000,1524.000000,1534.000000,1534.000000,1534.000000,1534.000000,1534.000000
mean,-0.096852,2625.398374,536.287093,1.434562e+05,11.968373,10.135053,7.341468,10.117851,4.340337e+06,4.425831e+06,...,80.967112,9.545267,2594.174967,1558.041142,39.473491,8.604348,1.117608,90.158521,9.841405,2.911191
std,0.739957,5942.483307,2196.450393,2.869863e+05,3.088631,2.824150,2.473902,2.905234,3.377434e+06,3.497115e+06,...,11.900026,5.240315,1083.200091,313.149226,30.890074,11.134773,0.995424,10.519099,10.518892,2.115077
min,-1.600000,0.000000,0.000000,0.000000e+00,5.650000,4.700000,3.200000,4.700000,1.444170e+05,1.525170e+05,...,38.660000,0.000000,1232.600000,988.700000,0.400000,0.050000,0.000000,58.459995,0.240151,0.240151
25%,-0.500000,102.250000,3.000000,9.650000e+03,9.540000,8.017500,5.697500,7.957500,2.052570e+06,1.918971e+06,...,74.570000,5.220000,2016.300000,1307.800000,15.200000,3.350000,0.590000,86.382550,2.742036,1.730658
50%,-0.300000,701.000000,168.000000,7.013500e+04,11.460000,9.465000,6.710000,9.410000,3.441587e+06,3.182069e+06,...,84.050000,9.080000,2380.000000,1528.600000,29.500000,5.280000,0.970000,95.164177,4.835823,2.090873
75%,0.300000,2880.000000,400.000000,1.500000e+05,13.900000,11.340000,8.590000,11.740000,6.023744e+06,7.132397e+06,...,89.810000,12.020000,2847.425000,1732.200000,54.600000,8.680000,1.460000,97.258336,13.617450,3.645862
max,2.300000,108653.000000,41788.000000,3.241437e+06,34.580000,32.140000,27.850000,31.290000,1.862066e+07,1.404697e+07,...,100.000000,40.250000,9856.500000,2220.700000,142.300000,100.000000,6.210000,99.759849,41.540005,10.294118


In [55]:
# Correlation matrix:
corr = outage.corr()
corr

# corr = outage.corr().stack()
# corr = corr[corr.index.get_level_values(0) != corr.index.get_level_values(1)]
# corr

,YEAR,MONTH,ANOMALY.LEVEL,OUTAGE.DURATION,DEMAND.LOSS.MW,CUSTOMERS.AFFECTED,RES.PRICE,COM.PRICE,IND.PRICE,TOTAL.PRICE,...,POPPCT_URBAN,POPPCT_UC,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND
YEAR,1.000000,-0.145328,0.152132,-0.144047,-0.048470,-0.203106,0.285305,0.178094,0.107126,0.203980,...,-0.059175,0.133157,-0.049423,-0.035008,0.006746,0.002862,0.050914,0.083103,-0.083108,-0.065317
MONTH,-0.145328,1.000000,0.122687,0.019649,0.019242,0.044265,0.040726,0.029512,0.038572,0.032534,...,0.028650,-0.026817,0.028900,0.066761,-0.026504,-0.024165,-0.038223,-0.056710,0.056706,-0.004792
ANOMALY.LEVEL,0.152132,0.122687,1.000000,0.008910,0.060534,0.012768,-0.002123,-0.021202,-0.045939,-0.019087,...,0.042433,0.015403,0.016084,0.052256,-0.077142,-0.054828,-0.049914,0.032730,-0.032731,-0.038350
OUTAGE.DURATION,-0.144047,0.019649,0.008910,1.000000,0.026798,0.261916,0.004394,-0.001193,-0.040322,0.001722,...,-0.005116,-0.049927,0.022115,-0.002205,0.038422,0.025040,-0.025940,-0.121614,0.121605,0.030389
DEMAND.LOSS.MW,-0.048470,0.019242,0.060534,0.026798,1.000000,0.523746,0.016727,0.060988,0.034917,0.046997,...,0.051795,-0.076131,0.051451,0.012847,0.019149,0.035769,-0.018441,-0.037732,0.037734,0.021429
CUSTOMERS.AFFECTED,-0.203106,0.044265,0.012768,0.261916,0.523746,1.000000,-0.020290,-0.011285,-0.008099,-0.005548,...,0.113629,-0.117916,0.093878,0.060508,-0.052409,0.007171,-0.066623,-0.009530,0.009527,0.020321
RES.PRICE,0.285305,0.040726,-0.002123,0.004394,0.016727,-0.020290,1.000000,0.927087,0.813129,0.964819,...,0.280213,-0.332752,0.416170,0.271060,0.242106,0.259328,0.151521,-0.190775,0.190781,0.014160
COM.PRICE,0.178094,0.029512,-0.021202,-0.001193,0.060988,-0.011285,0.927087,1.000000,0.863704,0.977913,...,0.323708,-0.364091,0.492072,0.330239,0.200140,0.279196,0.092000,-0.154140,0.154139,0.033239
IND.PRICE,0.107126,0.038572,-0.045939,-0.040322,0.034917,-0.008099,0.813129,0.863704,1.000000,0.888185,...,0.302756,-0.349632,0.329301,0.258182,0.259206,0.326347,0.152045,-0.164432,0.164422,0.064923
TOTAL.PRICE,0.203980,0.032534,-0.019087,0.001722,0.046997,-0.005548,0.964819,0.977913,0.888185,1.000000,...,0.337433,-0.404706,0.472762,0.281729,0.254493,0.322697,0.136429,-0.178898,0.178897,0.083094


In [56]:
outage.groupby(["YEAR", "U.S._STATE"])["RES.PERCEN"]

Major outages: depends on outage duration and # of people affected (at least 50,000 people)

In [57]:
# outage["OUTAGE.DURATION"].astype(float).unique()

In [58]:
# Getting rows of major outages:
outage.loc[(outage["CUSTOMERS.AFFECTED"] >= 50000) & (outage["OUTAGE.DURATION"])]

ValueError: cannot convert to 'bool'-dtype NumPy array with missing values. Specify an appropriate 'na_value' for this dtype.

### Assessment of Missingness

In [ ]:
# TODO
mask = outage.isna().sum(axis=0) > 0
columns_w_null = outage.columns[mask]
columns_w_null.shape

In [ ]:
columns_w_null

In [ ]:
outage.info()

In [ ]:
outage.shape

HURRICANE.NAMES: (MD) ; depends on if outage was due to hurricane


"NA" in the data file means that data wasn't available

# NMAR Analysis

- MONTH: MCAR


- CLIMATE.REGION: 
'ANOMALY.LEVEL', 
'CLIMATE.CATEGORY',
'CAUSE.CATEGORY.DETAIL',
'HURRICANE.NAMES',
'OUTAGE.DURATION',
'DEMAND.LOSS.MW', 'CUSTOMERS.AFFECTED', 'RES.PRICE', 'COM.PRICE',
       'IND.PRICE', 'TOTAL.PRICE', 'RES.SALES', 'COM.SALES', 'IND.SALES',
       'TOTAL.SALES', 'RES.PERCEN', 'COM.PERCEN', 'IND.PERCEN', 'POPDEN_UC',
       'POPDEN_RURAL', 'OUTAGE.START', 'OUTAGE.RESTORATION'


In [ ]:
outage["MONTH"].value_counts(dropna=False)

In [ ]:
outage["NERC.REGION"].value_counts(dropna=False)

In [ ]:
outage["CLIMATE.REGION"].value_counts(dropna=False)

In [ ]:
outage["CLIMATE.CATEGORY"].value_counts(dropna=False)